# Libraries

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import random
import time
from urllib.parse import quote
from datetime import datetime
import pytz

In [3]:
# https://mostaql.com/projects?keyword=علوم%20البيانات&budget_max=1000&sort=latest

### Create a New Requests Session

In [5]:
session = requests.Session()

### Set a New SOCKS5 Proxy with Tor
This function sets a new SOCKS5 proxy with random credentials

In [7]:
def set_new_proxy():
    port = 9150 
    credentials = f"{random.randint(10000, 99999)}:pass"
    proxy_url = f"socks5h://{credentials}@127.0.0.1:{port}"
    session.proxies = {'http': proxy_url,
                       'https': proxy_url}

In [8]:
def check_tor_ip():
        response = session.get("http://httpbin.org/ip", timeout=10)
        print("Tor IP:", response.text)
check_tor_ip()

Tor IP: {
  "origin": "102.189.170.226"
}



### Generate Search URL for Mostaql
Builds a Mostaql search URL using a given keyword

In [10]:
def generate_url(keyword):
    encoded_keyword = quote(keyword)
    url = f"https://mostaql.com/projects?keyword={encoded_keyword}&budget_max=1000&sort=latest"
    return url

### Fetch Job Data with Tor & Retry Logic
Sends a GET request to the provided URL using a Tor proxy

In [12]:
def get_job_data(url, max_retries=5):
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'X-Requested-With': 'XMLHttpRequest'}
    for attempt in range(max_retries):
        try:
            set_new_proxy()
            check_tor_ip()
            response = session.get(url, headers=headers, timeout=15)
            data = response.json()
            time.sleep(random.uniform(3, 6))
            return data['collection']
        except Exception as e:
            print(f"محاولة {attempt+1} فشلت: {e}")
            time.sleep(5)
    return []

### Parse Job Project Details from HTML
Extracts key details from a Mostaql project using BeautifulSoup

In [14]:
def parse_job_details(project):
    html = project['rendered']
    soup = BeautifulSoup(html, 'html.parser')

    project_id = project['id']

    title_tag = soup.find('a')
    title = title_tag.text.strip() if title_tag else 'غير متوفر'
    
    description_tag = soup.find('p')
    description = description_tag.text.strip() if description_tag else 'غير متوفر'
    
    date_tag = soup.find('time')
    if date_tag and 'datetime' in date_tag.attrs:
        utc_time_str = date_tag['datetime']
        utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S')
        utc_time = utc_time.replace(tzinfo=pytz.utc)
        local_time = utc_time.astimezone(pytz.timezone('Africa/Cairo'))
        date = local_time.strftime('%Y-%m-%d %H:%M:%S')
    else:
        date = 'غير متوفر'

    link = title_tag['href'] if title_tag else '#'
    
    return {
        'id': project_id,
        'title': title,
        'description': description,
        'date': date,
        'link': link
    }

### Extract Budget and Skills from Project Page

In [16]:
def extract_additional_details(project_url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = session.get(project_url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.content, 'html.parser')

        # الميزانية
        budget_tag = soup.find('span', dir='rtl')
        budget = budget_tag.text.strip() if budget_tag else 'غير متوفر'

        # المهارات
        skills_list = []
        skills_section = soup.find('ul', class_='skills')
        if skills_section:
            for li in skills_section.find_all('li'):
                skill = li.get_text(strip=True)
                skills_list.append(skill)

        return {
            'budget': budget,
            'skills': ', '.join(skills_list)
        }

    except Exception as e:
        print(f"فشل في جلب التفاصيل من {project_url}: {e}")
        return {
            'budget': 'غير متوفر',
            'skills': 'غير متوفر'
        }

### Scrape Mostaql Jobs by Keyword

In [18]:
def scrape_jobs(keyword):
    url = generate_url(keyword)
    projects = get_job_data(url)

    if projects:
        job_list = []
        for project in projects:
            job_details = parse_job_details(project)
            extra_details = extract_additional_details(job_details['link'])
            job_details.update(extra_details)
            job_list.append(job_details)
        
        df = pd.DataFrame(job_list)
        print(df.head())
        df.to_csv(f"mostaql_jobs_{keyword}.csv", index=False)
        return df

## هتدخل اسم الوظيفة بالعربي هنا

In [20]:
scrape_jobs("علوم البيانات")

Tor IP: {
  "origin": "185.220.101.101"
}

        id                                              title  \
0  1076749  متخصص تقييم ومتابعة وضمان جودة (QA) لمنصة إلكت...   
1  1025445                                 تحسين محركات البحث   
2  1025025    إطلاق حملات إعلانية وإدارة حسابات تواصل إجتماعي   
3  1076722                   تصميم Wizrd (UI/UX) من عدة خطوات   
4  1076711                  خدمة عملاء واتس اب لمتجر إلكتروني   

                                         description                 date  \
0  نبحث عن شخص متمكن للانضمام إلى الفريق للقيام ب...  2025-04-19 12:16:26   
1  مطلوب اشخاص لديهم خبرة لا تقل عن 5 سنوات ١- كت...  2025-04-19 11:45:06   
2  مطلوب اشخاص لديهم خبرة لا تقل عن 5 سنوات ١- اط...  2025-04-19 11:45:00   
3  المطلوب تصميم Wizard مكون من عدة خطوات في كل خ...  2025-04-19 11:34:31   
4  نبحث عن موظف خدمة عملاء محترف، لديه فهم للسوق ...  2025-04-19 11:17:22   

                                                link             budget  \
0  https://mostaql.com/proje

,id,title,description,date,link,budget,skills
0,1076749,متخصص تقييم ومتابعة وضمان جودة (QA) لمنصة إلكت...,نبحث عن شخص متمكن للانضمام إلى الفريق للقيام ب...,2025-04-19 12:16:26,https://mostaql.com/project/1076749-%D9%85%D8%...,$100.00 - $250.00,"برمجة مواقع, برمجة التطبيقات, برمجة قواعد البي..."
1,1025445,تحسين محركات البحث,مطلوب اشخاص لديهم خبرة لا تقل عن 5 سنوات ١- كت...,2025-04-19 11:45:06,https://mostaql.com/project/1025445-%D8%AA%D8%...,$250.00 - $500.00,"كتابة مقالات, كتابة المحتوى, تحسين محركات البح..."
2,1025025,إطلاق حملات إعلانية وإدارة حسابات تواصل إجتماعي,مطلوب اشخاص لديهم خبرة لا تقل عن 5 سنوات ١- اط...,2025-04-19 11:45:00,https://mostaql.com/project/1025025-%D8%A5%D8%...,$250.00 - $500.00,"التسويق عبر البريد الإلكتروني, إدارة حسابات ال..."
3,1076722,تصميم Wizrd (UI/UX) من عدة خطوات,المطلوب تصميم Wizard مكون من عدة خطوات في كل خ...,2025-04-19 11:34:31,https://mostaql.com/project/1076722-%D8%AA%D8%...,$50.00 - $100.00,"تصميم واجهة المستخدم, التصميم الإبداعي, تصميم ..."
4,1076711,خدمة عملاء واتس اب لمتجر إلكتروني,نبحث عن موظف خدمة عملاء محترف، لديه فهم للسوق ...,2025-04-19 11:17:22,https://mostaql.com/project/1076711-%D8%AE%D8%...,$50.00 - $100.00,"إدارة علاقات العملاء, خدمة العملاء, دعم مكتب ا..."
5,1075601,تصميم الفيلا وشقتين مدخل مستقل - الأردن,بيانات تصميم الفيلا قطعة الأرض رقم 569. مساحة ...,2025-04-19 11:02:57,https://mostaql.com/project/1075601-%D8%AA%D8%...,$50.00 - $100.00,"تصميم مخططات معمارية, مخططات فلل, تصميم 2d, أو..."
6,1076343,مراجعة مع باحث CFA,مراجعه وتنشيط باحث في CFA المطلوب حاصل على CFA...,2025-04-19 10:42:53,https://mostaql.com/project/1076343-%D9%85%D8%...,$50.00 - $100.00,"التحليل المالي, الإدارة المالية, تحليل الأسواق..."
7,1058130,موظف خدمة عملاء واتس اب لمتجر إلكتروني,نبحث عن موظف خدمة عملاء محترف، لديه فهم للسوق ...,2025-04-19 10:36:18,https://mostaql.com/project/1058130-%D9%85%D9%...,$50.00 - $100.00,"خدمة العملاء, خدمة العملاء عبر الهاتف, إدارة ع..."
8,1076696,مصمم ui ux لتصميم أجزاء من صفحات موقع,مرحبا ايها الرائعون كيف الحال ، مطلوب مصمم ui ...,2025-04-19 10:24:11,https://mostaql.com/project/1076696-%D9%85%D8%...,$25.00 - $50.00,"تصميم جرافيك, تصميم UI UX, واجهة برمجة التطبيق..."
9,1076596,مستقل محترف لتنسيق توريد منتج فاخر إلى منصات ا...,نبحث عن مستقل متمرس يتولى مشروعا متكاملا لتوسي...,2025-04-19 10:17:45,https://mostaql.com/project/1076596-%D9%85%D8%...,$100.00 - $250.00,"الكتابة الإبداعية, إدارة أعمال, إدارة المشاريع..."
